In [16]:
import numpy as np

# Outils sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Outils pytorch
import torch # package Racine
from torch import nn # Réseau de Neurones (Neural Network)
import torch.nn.functional as F # Couches, fonctions d'activation ...
import torch.autograd as autograd # Calcul dérivée (Gradient)
import torch.optim as optim # Optimiser pour la descente de Gradient

# Librairie graphique plotly
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff

dir(optim)

['ASGD',
 'Adadelta',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'LBFGS',
 'Optimizer',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'lr_scheduler']

## Construction d'une classe module

In [ ]:
class ClasseModele(nn.Module):
    def __init__(self, input_dim):
        super(ClasseModele, self).__init__()
        
        self.layer1 = nn.Linear(input_dim,50) # Première couche de 50 neurones
        self.layer2 = nn.Linear(50, 20)       # deuxième couche de 20 neurones        
        self.layer3 = nn.Linear(20, 3)        # Couche de sortie de 3 neurones (3 classes)
        
    def forward(self, x):
        # https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))        
        x = F.softmax(self.layer3(x),dim=1) # Application de la fonction sofmax à la couche de sortie
        return x    

## Chargement des données Iris

In [4]:
features, labels = load_iris(return_X_y=True)
print(features.shape,labels.shape)
iris = np.concatenate((features, labels.reshape([150,1])), axis=1)
iris[np.random.randint(len(iris), size=10)]

(150, 4)(150,)


array([[5. , 3.4, 1.6, 0.4, 0. ],
       [4.9, 3.1, 1.5, 0.2, 0. ],
       [5.8, 2.6, 4. , 1.2, 1. ],
       [5.6, 2.7, 4.2, 1.3, 1. ],
       [6. , 3.4, 4.5, 1.6, 1. ],
       [6.4, 3.1, 5.5, 1.8, 2. ],
       [6.8, 2.8, 4.8, 1.4, 1. ],
       [4.7, 3.2, 1.6, 0.2, 0. ],
       [4.6, 3.6, 1. , 0.2, 0. ],
       [5.5, 2.4, 3.7, 1. , 1. ]])

## Création d'un set de train et d'un set de test (dev)

In [5]:
features_train,features_dev, labels_train, labels_dev = train_test_split(features, labels, random_state=42)
print(features_train.shape,features_dev.shape, labels_train.shape, labels_dev.shape)
labels_dev[:10]

(112, 4)(38, 4)(112,)(38,)


array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1])

## Apprentissage avec PyTorch

In [6]:
# Création d'un objet Modèle
model = ClasseModele(features_train.shape[1])

# choix de l'algorithme de Descente de Gradient et du learning Rate
# https://pytorch.org/docs/stable/optim.html#algorithms
optimizer = optim.Adam(model.parameters(), lr=0.01)

# choix de la fonction de coût
# https://pytorch.org/docs/stable/nn.html#loss-functions
loss_fn = nn.CrossEntropyLoss()

#Nombre d'itération sur les données
epochs = 100

print(model)

ClasseModele(
  (layer1): Linear(in_features=4, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=20, bias=True)
  (layer3): Linear(in_features=20, out_features=3, bias=True)
)


In [7]:
x_train, y_train = torch.from_numpy(features_train).float(), torch.from_numpy(labels_train).long()
for epoch in range(1, epochs+1):    
    y_pred = model(x_train)
    loss = loss_fn(y_pred, y_train)
    if not epoch%10 :        
        print('Epoch #%i Loss=%.2f'%(epoch,loss.item()))                
    
    optimizer.zero_grad() # Réinitialise le gradient
    loss.backward()       # Exécute la backpropagation
    optimizer.step()      # Met à jours les paramètres du réseau

Epoch #10 Loss=0.86
Epoch #20 Loss=0.73
Epoch #30 Loss=0.62
Epoch #40 Loss=0.59
Epoch #50 Loss=0.58
Epoch #60 Loss=0.58
Epoch #70 Loss=0.58
Epoch #80 Loss=0.58
Epoch #90 Loss=0.58
Epoch #100 Loss=0.58


## Prédiction

In [8]:
#Création du Tensor x_dev, de type float
x_dev = torch.from_numpy(features_dev).float()
#Exécution du modèle
pred = model(x_dev)
# detach() sort le calcul de la prediction du calcul du gradient, et conversion en tableau numpy
print(pred.grad_fn)
pred = pred.detach()
print(pred.grad_fn)
pred = pred.numpy()
pred

None


array([[1.1873593e-05, 9.9841297e-01, 1.5750958e-03],
       [9.9999011e-01, 9.8745177e-06, 3.9618801e-29],
       [2.0332182e-18, 5.7576144e-10, 1.0000000e+00],
       [1.3627196e-05, 9.9698752e-01, 2.9987968e-03],
       [5.4298266e-06, 9.9940932e-01, 5.8523996e-04],
       [9.9997461e-01, 2.5387912e-05, 4.3657088e-27],
       [3.7792514e-04, 9.9962211e-01, 2.6324978e-08],
       [1.6656740e-08, 2.1293502e-02, 9.7870654e-01],
       [2.0658404e-06, 3.5863549e-01, 6.4136249e-01],
       [8.4655592e-05, 9.9991453e-01, 8.1228063e-07],
       [2.4508168e-07, 1.5070219e-01, 8.4929758e-01],
       [9.9995351e-01, 4.6534267e-05, 3.5393820e-25],
       [9.9999452e-01, 5.4516645e-06, 2.2544263e-29],
       [9.9995196e-01, 4.8046757e-05, 2.0235807e-25],
       [9.9999011e-01, 9.8812343e-06, 5.2594638e-28],
       [1.0226538e-05, 9.9976045e-01, 2.2935867e-04],
       [8.5010205e-14, 5.4382701e-07, 9.9999940e-01],
       [8.4136562e-05, 9.9990726e-01, 8.5264792e-06],
       [1.8436780e-05, 9.940

## Calcul du score de précision

### Transformation des probabilités en résultat unique

In [9]:
prevision_type_iris =  np.argmax(model(x_dev).detach().numpy(),axis=1)
prevision_type_iris

array([1, 0, 2, 1, 1, 0, 1, 2, 2, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0], dtype=int64)

### Calcul de la précision

In [10]:
print ("Précision de la prévision = %.1f%% "%(accuracy_score(labels_dev, prevision_type_iris)*100))
print('%i prévision(s) correcte(s), %i erreur(s)'%(sum(prevision_type_iris==labels_dev),sum(prevision_type_iris!=labels_dev)))

Précision de la prévision = 97.4%
37 prévision(s) correcte(s), 1 erreur(s)


### Comparaison Courbes 

In [13]:
acc_train=[]
acc_dev=[]
tab_epoch=[]
new_model = ClasseModele(features_train.shape[1])
new_optimizer = optim.Adam(new_model.parameters(), lr=0.01)
epochs=100
for epoch in range(1, epochs+1):    
    y_pred = new_model(x_train)
    loss = loss_fn(y_pred, y_train)
    
    if not epoch%10 :                
        print('Epoch #%i Loss=%.2f'%(epoch,loss.item()))        
        tab_epoch.append(epoch)
        acc_dev.append(
            accuracy_score(
                labels_dev,
                np.argmax(new_model(x_dev).detach().numpy(),axis=1)
            )*100            
        )
        acc_train.append(
            accuracy_score(
                labels_train,
                np.argmax(new_model(x_train).detach().numpy(),axis=1)
            )*100
        )           
    new_optimizer.zero_grad() # Réinitialise le gradient
    loss.backward()       # Exécute la backpropagation
    new_optimizer.step()      # Met à jours les paramètres du réseau

print(acc_train,acc_dev)

Epoch #10 Loss=0.95
Epoch #20 Loss=0.79
Epoch #30 Loss=0.65
Epoch #40 Loss=0.59
Epoch #50 Loss=0.58
Epoch #60 Loss=0.58
Epoch #70 Loss=0.58
Epoch #80 Loss=0.58
Epoch #90 Loss=0.58
Epoch #100 Loss=0.58
[65.17857142857143, 86.60714285714286, 98.21428571428571, 98.21428571428571, 98.21428571428571, 98.21428571428571, 98.21428571428571, 98.21428571428571, 98.21428571428571, 98.21428571428571][71.05263157894737, 84.21052631578947, 97.36842105263158, 100.0, 97.36842105263158, 97.36842105263158, 97.36842105263158, 97.36842105263158, 97.36842105263158, 97.36842105263158]


## Graphique

In [14]:
train_line =go.Scatter(x=tab_epoch,y=acc_train,name='Train set')
dev_line =go.Scatter(x=tab_epoch,y=acc_dev,name='Dev set')

layout = go.Layout(title="Comparaison Précision Train/Test",titlefont=dict(size=40),autosize=False, width=1100,height=1100)

data=[train_line,dev_line]
fig = go.Figure(data=data, layout=layout)
fig.show()